# InstallingTransformers

In [2]:
!pip install simpletransformers

     -------------------------------------- 250.8/250.8 kB 1.4 MB/s eta 0:00:00
     -------------------------------------- 977.5/977.5 kB 2.1 MB/s eta 0:00:00
     -------------------------------------- 519.3/519.3 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.1/2.1 MB 1.5 MB/s eta 0:00:00
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached safetensors-0.3.2-cp310-cp310-win_amd64.whl (266 kB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 215.6/215.6 kB 1.5 MB/s eta 0:00:00
     ------------------------------------ 134.8

In [3]:
import re

In [4]:
from simpletransformers.language_modeling import LanguageModelingModel, LanguageModelingArgs

# Model Fine-Tunning

In [6]:
model_config=LanguageModelingArgs()

In [7]:
model_config.reprocess_input_data=True
model_config.overwrite_output_dir=True
model_config.num_train_epochs=1
model_config.best_model_dir="output/best_model"
model_config.save_best_model=True
model_config.train_batch_size=1
model_config.dataset_type="simple"
model_config.mlm=False
model_config.vocab_size=50257

In [9]:
with open('nlp.txt','r') as fp:
    text = fp.read().split('\n')

In [10]:
text

['Natural language processing (NLP) is an interdisciplinary subfield of linguistics and computer science. It is primarily concerned with processing natural language datasets, such as text corpora or speech corpora, using either rule-based or probabilistic (i.e. statistical and, most recently, neural network-based) machine learning approaches. The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them. The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.',
 'Natural language processing has its roots in the 1950s. Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, though at the time that was not articulated as a problem separate from artificial intelligence. The proposed test includes

In [12]:
text[2].split('.')

['Up to the 1980s, most natural language processing systems were based on complex sets of hand-written rules',
 ' Starting in the late 1980s, however, there was a revolution in natural language processing with the introduction of machine learning algorithms for language processing',
 " This was due to both the steady increase in computational power (see Moore's law) and the gradual lessening of the dominance of Chomskyan theories of linguistics (e",
 'g',
 ' transformational grammar), whose theoretical underpinnings discouraged the sort of corpus linguistics that underlies the machine-learning approach to language processing',
 '']

# Text Preprocessing 

In [13]:
bag = [txt for sentence in text for txt in sentence.split('.') if txt!='']
bag_size=len(bag)

In [14]:
bag_size

14

In [15]:
import random

In [16]:
train_sentence=[]
test_sentence=[]
label=[]
for p in text:
    sentences=[
        sentence for sentence in p.split('.') if sentence!=''
    ]
    num_sentence = len(sentences)
    if num_sentence>1:
        start = random.randint(0,num_sentence-2)
        if random.random()>=0.5:
            train_sentence.append(sentences[start])
            test_sentence.append(sentences[start+1])
            label.append(0)
        else:
            index=random.randint(0,bag_size-1)
            train_sentence.append(sentences[start])
            test_sentence.append(bag[index])
            label.append(1)

In [18]:
for i in range(2):
    print(label[i])
    print(train_sentence[i] + '\n---')
    print(test_sentence[i] + '\n---')

0
e
---
 statistical and, most recently, neural network-based) machine learning approaches
---
1
 Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, though at the time that was not articulated as a problem separate from artificial intelligence
---
g
---


In [19]:
len(train_sentence)

3

In [20]:
len(test_sentence)

3

In [21]:
def textCleaning(s):
    s = str(s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace("[\w*"," ")
    return s

# Test-Train Split

In [22]:
train_data = open('Train_data.txt','w')
text_data=textCleaning(train_sentence)
train_data.write(text_data)
train_data.close()

In [23]:
test_data = open('Test_data.txt','w')
text_data=textCleaning(test_sentence)
test_data.write(text_data)
test_data.close()

In [24]:
train_file='Train_data.txt'
test_file='Test_data.txt'

In [25]:
model=LanguageModelingModel(
    "gpt2","gpt2",args=model_config, train_files=train_file,use_cuda=False
)

C:\Users\Dell\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\cache_dir. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# Model Training

In [26]:
model.train_model(train_file,eval_first=test_file)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1 [00:00<?, ?it/s]

(1, 4.710568428039551)

In [28]:
eval_score=model.eval_model(test_file)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
eval_score

{'eval_loss': 4.886779308319092, 'perplexity': tensor(132.5261)}

In [30]:
from simpletransformers.language_generation import LanguageGenerationModel, LanguageGenerationArgs

In [31]:
lang_gen_args = LanguageGenerationArgs()
lang_gen_args.max_length=1

In [34]:
# from simpletransformers.language_generation.language_generation_model import MAX_LENGTH
model = LanguageGenerationModel("gpt2","outputs/checkpoint-1-epoch-1",use_cuda=False)
output = model.generate("Have a nice ")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# Importing GPT-2 Model

In [35]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [36]:
model_name="gpt2-medium"

In [37]:
model=GPT2LMHeadModel.from_pretrained(model_name)
tokenizer=GPT2Tokenizer.from_pretrained(model_name,pad_token='<PAD>')

C:\Users\Dell\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [38]:
tokenizer.add_special_tokens({'pad_token':'[PAD]'})

1

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [40]:
tokenized_train_data = [tokenizer.encode(text, return_tensors="pt").to(device) for text in train_file]

In [41]:
text='Hows your day '
text_ids=tokenizer.encode(text, return_tensors="pt").to(device)

In [42]:
outputs = model.generate(text_ids, max_length=10, num_return_sequences=1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [43]:
generated_text=tokenizer.decode(outputs[0],skip_special_tokens=True)

In [44]:
generated_text.replace('\xa0',"")
generated_text.replace('\n',"")

'Hows your day?""I'

In [45]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [46]:
def load_model(model_name):
  model=GPT2LMHeadModel.from_pretrained(model_name)
  return model

In [47]:
def load_tokenizer(tokenizer_path):
  tokenizer=GPT2Tokenizer.from_pretrained(tokenizer_path)
  return tokenizer

# Model Generation

In [48]:
def generate_text(sequence, max_length):
  model_name="gpt2-medium"
  model=load_model(model_name)
  tokenizer=load_tokenizer(model_name)
  ids=tokenizer.encode(f"{sequence}", return_tensors='pt')
  final_outputs=model.generate(
      ids,
      do_sample=True,
      max_length=max_length,
      pad_token_id=model.config.eos_token_id,
      top_k=50,
      top_p=0.95)
  print(tokenizer.decode(final_outputs[0],skip_special_tokens=True))

In [50]:
sequence=input()
max_len=int(input())
generate_text(sequence,max_len)

have a nice
8
have a nice family, some wonderful neighbors


In [52]:
validation_texts=['Natural language processing (NLP) is an interdisciplinary subfield of linguistics and computer science.The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them.']

# Perplexity Score

In [53]:
perplexities=[]
for text in validation_texts:
  input_ids=tokenizer.encode(text,return_tensors='pt')
  with torch.no_grad():
    loss=model(input_ids,labels=input_ids).loss
    perplexities.append(torch.exp(loss))

average_perplexity = sum(perplexities)/len(perplexities)
print("Average Perplexity: ", average_perplexity.item())

Average Perplexity:  15.462457656860352
